In [68]:
import numpy as np
import pandas as pd
import soundfile as sf
from IPython.display import display, Audio
import matplotlib.pyplot as plt
%matplotlib inline

import os


# SR = 16_000
SR = 8_000


def adisplay(audio):
    display(Audio(audio, rate=SR))

In [9]:
ls /home/alexnikko/data/Libri2Mix/wav8k/min/metadata/

metrics_test_mix_both.csv    mixture_test_mix_both.csv
metrics_test_mix_clean.csv   mixture_test_mix_clean.csv
metrics_test_mix_single.csv  mixture_test_mix_single.csv


In [10]:
df = pd.read_csv('/home/alexnikko/data/Libri2Mix/wav8k/min/metadata/metrics_test_mix_clean.csv')
df.head()

,mixture_ID,source_1_SNR,source_2_SNR
0,4077-13754-0001_5142-33396-0065,1.196426,-1.196427
1,6930-76324-0027_5683-32879-0011,0.394409,-0.394412
2,1188-133604-0025_4992-23283-0016,0.546560,-0.546560
3,5683-32866-0012_121-121726-0001,4.447866,-4.447868
4,1995-1837-0025_5142-36377-0017,-1.642787,1.642786


In [11]:
df = pd.read_csv('/home/alexnikko/data/Libri2Mix/wav8k/min/metadata/mixture_test_mix_clean.csv')
df.head()

,mixture_ID,mixture_path,source_1_path,source_2_path,length
0,4077-13754-0001_5142-33396-0065,/home/alexnikko/data/Libri2Mix/wav8k/min/test/...,/home/alexnikko/data/Libri2Mix/wav8k/min/test/...,/home/alexnikko/data/Libri2Mix/wav8k/min/test/...,30160
1,6930-76324-0027_5683-32879-0011,/home/alexnikko/data/Libri2Mix/wav8k/min/test/...,/home/alexnikko/data/Libri2Mix/wav8k/min/test/...,/home/alexnikko/data/Libri2Mix/wav8k/min/test/...,66160
2,1188-133604-0025_4992-23283-0016,/home/alexnikko/data/Libri2Mix/wav8k/min/test/...,/home/alexnikko/data/Libri2Mix/wav8k/min/test/...,/home/alexnikko/data/Libri2Mix/wav8k/min/test/...,35960
3,5683-32866-0012_121-121726-0001,/home/alexnikko/data/Libri2Mix/wav8k/min/test/...,/home/alexnikko/data/Libri2Mix/wav8k/min/test/...,/home/alexnikko/data/Libri2Mix/wav8k/min/test/...,47400
4,1995-1837-0025_5142-36377-0017,/home/alexnikko/data/Libri2Mix/wav8k/min/test/...,/home/alexnikko/data/Libri2Mix/wav8k/min/test/...,/home/alexnikko/data/Libri2Mix/wav8k/min/test/...,37480


In [34]:
df.iloc[0].source_1_path

'/home/alexnikko/data/Libri2Mix/wav8k/min/test/s1/4077-13754-0001_5142-33396-0065.wav'

In [70]:
adisplay(df.iloc[0].source_1_path)

In [35]:
print(df.iloc[0].source_2_path)
adisplay(df.iloc[0].source_2_path)

/home/alexnikko/data/Libri2Mix/wav8k/min/test/s2/4077-13754-0001_5142-33396-0065.wav


In [36]:
print(df.iloc[0].mixture_path)
adisplay(df.iloc[0].mixture_path)

/home/alexnikko/data/Libri2Mix/wav8k/min/test/mix_clean/4077-13754-0001_5142-33396-0065.wav


In [38]:
from torchmetrics.functional import signal_noise_ratio

In [40]:
row = df.iloc[0]
s1, _ = sf.read(row.source_1_path)
s2, _ = sf.read(row.source_2_path)
mixture, _ = sf.read(row.mixture_path)

In [43]:
import torch

In [44]:
def to_torch(x):
    return torch.from_numpy(x)

In [45]:
signal_noise_ratio(to_torch(mixture), to_torch(s1))

tensor(1.1964, dtype=torch.float64)

In [46]:
signal_noise_ratio(to_torch(mixture), to_torch(s2))

tensor(-1.1965, dtype=torch.float64)

In [47]:
!pip install speechbrain

     |████████████████████████████████| 498 kB 1.4 MB/s eta 0:00:01     |███████████████████             | 296 kB 1.4 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 11.3 MB/s eta 0:00:01
     |████████████████████████████████| 120 kB 12.6 MB/s eta 0:00:01
     |████████████████████████████████| 661 kB 11.1 MB/s eta 0:00:01
     |████████████████████████████████| 109 kB 12.0 MB/s eta 0:00:01
     |████████████████████████████████| 539 kB 11.2 MB/s eta 0:00:01
  Created wheel for hyperpyyaml: filename=HyperPyYAML-1.0.1-py3-none-any.whl size=15192 sha256=ce977e95f96c12c018c7588a09720d669ea1e7cc731eb0761cf2453e57775d9b
  Stored in directory: /home/alexnikko/.cache/pip/wheels/c2/bb/de/befc66499e9c2b4b8b163e018abcace61c80220e6a28950a65
Successfully built hyperpyyaml


In [48]:
from speechbrain.pretrained import SepformerSeparation as separator
import torchaudio

In [49]:
source="speechbrain/sepformer-libri2mix"
savedir='pretrained_models/sepformer-libri2mix'
model = separator.from_hparams(source=source, savedir=savedir)

Downloading:   0%|          | 0.00/4.59k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/17.3k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/113M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/17.3k [00:00<?, ?B/s]

In [54]:
preds = model(to_torch(mixture)[None].float())

In [55]:
preds.shape

torch.Size([1, 30160, 2])

In [56]:
preds = preds.permute(0, 2, 1)  # b, spk, T

In [57]:
from torchmetrics.functional import permutation_invariant_training

In [62]:
target = torch.stack((to_torch(s1), to_torch(s2)))[None]
target.shape

torch.Size([1, 2, 30160])

In [64]:
permutation_invariant_training(preds, target, metric_func=signal_noise_ratio)

(tensor([-32.4939], dtype=torch.float64), tensor([[0, 1]]))

In [71]:
adisplay(preds[0, 0])

In [72]:
adisplay(preds[0, 1])

In [81]:
from torchmetrics.functional import scale_invariant_signal_noise_ratio as SISNR, signal_distortion_ratio as SDR

In [76]:
permutation_invariant_training(preds, target, metric_func=SISNR)

(tensor([17.9773], dtype=torch.float64), tensor([[0, 1]]))

In [78]:
SISNR(preds[0, 0], target[0, 0])

tensor(18.5890, dtype=torch.float64)

In [79]:
SISNR(preds[0, 1], target[0, 1])

tensor(17.3655, dtype=torch.float64)

In [80]:
(SISNR(preds[0, 1], target[0, 1]) + SISNR(preds[0, 0], target[0, 0])) / 2

tensor(17.9773, dtype=torch.float64)

In [87]:
SISNR(to_torch(mixture)[None, None].expand_as(target), target)

tensor(-0.0483, dtype=torch.float64)

In [89]:
device = 'cuda:0'

In [90]:
from tqdm.auto import tqdm

In [97]:
model.eval()
model.to(device);

In [103]:
model.device = device

In [122]:
metrics = []
for row in tqdm(df.itertuples(), total=len(df)):
    s1, _ = sf.read(row.source_1_path)
    s2, _ = sf.read(row.source_2_path)
    mixture, _ = sf.read(row.mixture_path)
    
    s1, s2, mixture = [to_torch(x).float() for x in [s1, s2, mixture]]
    target = torch.stack((s1, s2))[None]
    
    
    with torch.inference_mode():
        preds = model(mixture[None].to(device))
    preds = preds.cpu().permute(0, 2, 1)
    
    sisnr, perm = permutation_invariant_training(preds, target, metric_func=SISNR)
    sisnr_inp = SISNR(mixture[None, None].expand_as(target), target).mean()
    
    sdr, perm = permutation_invariant_training(preds, target, metric_func=SDR)
    sdr_inp = SDR(mixture[None, None].expand_as(target), target).mean()
    
    metrics.append({
        'SI-SNR_inp': sisnr_inp.item(),
        'SI-SNR': sisnr.item(),
        'SI-SNRi': sisnr.item() - sisnr_inp.item(),
        'SDR_inp': sdr_inp.item(),
        'SDR': sdr.item(),
        'SDRi': sdr.item() - sdr_inp.item(),
    })

  0%|          | 0/3000 [00:00<?, ?it/s]

In [124]:
pd.DataFrame(metrics).mean()

SI-SNR_inp     0.000514
SI-SNR        20.582695
SI-SNRi       20.582181
SDR_inp        0.179233
SDR           21.046906
SDRi          20.867673
dtype: float64

In [118]:
sisnr, sisnr_inp, sdr, sdr_inp

(tensor([17.9781]), tensor(-0.0483), tensor([18.4959]), tensor(0.0444))

In [121]:
for key, value in metrics[0].items():
    print(key, value)

SI-SNR_inp -0.04831784963607788
SI-SNR 17.978103637695312
SI-SNRi 18.02642148733139
SDR_inp 0.04440373182296753
SDR 18.495933532714844
SDRi 18.451529800891876


In [127]:
for key, value in pd.DataFrame(metrics).mean().to_dict().items():
    print(key, value)

SI-SNR_inp 0.0005142472969988981
SI-SNR 20.58269546131293
SI-SNRi 20.582181214015932
SDR_inp 0.17923308291410406
SDR 21.046906365185976
SDRi 20.867673282271873
